* OpenAI Whisper 可以把影片加字幕，但研究、訪談採取逐字稿的情景則不太合用，因無法辨識說話者，不會加標點。
* 這個程式用 Whisper 把語音轉正體中文逐字稿、每句結束加上全形逗號（但只會加逗號），再儲存成「原檔名_output.txt」
* 你可把文字檔下載後，截去不需要的部分、分段、加上合適標點（句號、驚歎號等）。

# 安裝及檢查必要的程式庫

執行每一格都按下「shift + Enter」它會執行這一格並跳到下一格，完成執行後左方會顯示數字而不是箭頭，這些是使用到的程式庫，如果沒安裝，它檢查完會去安裝，如果已經安裝了很快就跳到下一格。

* Whisper 聽打需要安裝 Whisper 以及 ffmpeg。
* Whisper 基本會轉換簡中，用 OpenCC 轉成正題中文。
* 測試過用 C++ 寫的 opencc，某些依賴項放在 OS 中有可能找不到，用 opencc_python_reimplemented 解決此問題。

In [ ]:
!pip install -U --quiet openai-whisper

In [ ]:
!pip install --quiet ffmpeg-python

In [ ]:
!pip install --quiet opencc_python_reimplemented

In [ ]:
!pip install --quiet pydub

# 找到要轉檔的檔案路徑

存入你要轉檔的原檔案，通常是 MP3 格式，如果不是 MP3，它會自動轉換 WAV（錄音筆）, MP4（影音檔）, M4A（iPhone/ Mac 錄音檔） 這幾種常用格式，其他沒測過。

* 如果你在本機使用 Anaconda/ JupyterLab，就把檔案放在檔案夾；
* 如果在 Colab 就要在側邊欄上傳檔案。 

確定檔案擺好後執行這一格，找到你的檔案路徑（例如：./filename.mp3），複製下來。

In [ ]:
# 獲得相對路徑
import os
from pathlib import Path

# Root dir
root_directory = Path(".")

# travel all the dirs/ files
for root, directories, files in os.walk(root_directory):
    # 输出当前目录路径
    current_dir = Path(root)
    print("Current Dir:", current_dir)

    # 输出当前目录中的文件
    for file in files:
        file_path = current_dir / file
        print("File:", file_path.as_posix())

# 進行聽打

In [ ]:
import whisper
import time
from IPython.display import display, clear_output
import os
import opencc  # 用於簡繁體轉換的庫
from pydub import AudioSegment

def convert_to_mp3(input_file):
    base_name = os.path.splitext(input_file)[0]
    output_file = f"{base_name}.mp3"
    
    # 如果已是 mp3，就不轉換
    if input_file.lower().endswith('.mp3'):
        return input_file
    
    # 轉換為 mp3
    audio = AudioSegment.from_file(input_file)
    audio.export(output_file, format="mp3")
    
    return output_file

# 定義輸入文件
input_file = r"./240612_0939_ilan_GongCeHui.mp4"  # 示例文件路径

# 不管輸入是何種格式都轉成 mp3
converted_file = convert_to_mp3(input_file)

# 載入 Whisper 模型
model = whisper.load_model("medium")  # small / medium / large

# 進行聽打，指定語言為中文（不然它會翻譯成英文）
try:
    result = model.transcribe(converted_file, language="zh")  # 使用转换后的文件路径
except Exception as e:
    clear_output(wait=True)
    display(f"識別過程中出現錯誤: {str(e)}")
    raise

# 計算總的執行時間
end_time = time.time()
elapsed_time = end_time - start_time
display(f"識別過程共花費了 {elapsed_time:.2f} 秒")

# 獲取轉錄的文本和分段信息
recognized_text = result["text"]
segments = result.get("segments", [])

# 用 opencc 將簡中轉換為繁中（default 是簡中）
converter = opencc.OpenCC('s2t')
recognized_text = converter.convert(recognized_text)

# 在每個段落結尾添加中文全形逗號
new_recognized_text = ""
for segment in segments:
    # 獲取段落文本
    segment_text = segment["text"]
    # 將段落文本轉換為繁體中文
    segment_text = converter.convert(segment_text)
    # 在段落尾部添加全形逗號
    new_recognized_text += segment_text + "，"

# 打印添加全形逗號後的文本
print("添加全形逗號後的文本：")
print(new_recognized_text)

# 生成輸出文件路徑
input_file_name = os.path.basename(input_file)  # 獲取輸入文件名
input_file_name_without_ext = os.path.splitext(input_file_name)[0]  # 去除擴展名
output_file_name = f"{input_file_name_without_ext}_output.txt"  # 生成輸出文件名
output_file_path = os.path.join(os.path.dirname(input_file), output_file_name)  # 生成完整的輸出文件路徑

# 將文本保存到 txt 文件
with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(new_recognized_text)

print("文本已保存到", output_file_path)

# 下載完成的文字檔
最後，從左側邊欄的檔案夾符號裡下載「原檔名_output.txt」即可使用。